# Экономика продукта

# Задача 1.

Начнём с выручки — наиболее общего показателя, который покажет, какой доход приносит наш сервис.

Задание:

Для каждого дня в таблице orders рассчитайте следующие показатели:

Выручку, полученную в этот день.
Суммарную выручку на текущий день.
Прирост выручки, полученной в этот день, относительно значения выручки за предыдущий день.
Колонки с показателями назовите соответственно revenue, total_revenue, revenue_change. Колонку с датами назовите date.

Прирост выручки рассчитайте в процентах и округлите значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: date, revenue, total_revenue, revenue_change

Пояснение: 

Будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

Суммарная выручка на текущий день — это результат сложения выручки, полученной в текущий день, со значениями аналогичного показателя всех предыдущих дней.

При расчёте выручки помните, что не все заказы были оплачены — некоторые были отменены пользователями.

Не забывайте при делении заранее приводить значения к нужному типу данных. Пропущенные значения прироста для самой первой даты не заполняйте — просто оставьте поля в этой строке пустыми.

```sql
SELECT date,
       revenue,
       sum(revenue) OVER(ORDER BY date) as total_revenue,
       round(revenue / (lag(revenue, 1) OVER(ORDER BY date)::decimal) * 100 - 100,
             2) as revenue_change
FROM   (SELECT date,
               sum(price) as revenue
        FROM   (SELECT order_id,
                       creation_time::date as date,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) as t1
            LEFT JOIN products using(product_id)
        GROUP BY date) as t11
ORDER BY date
```

# Задача 2.

Теперь на основе данных о выручке рассчитаем несколько относительных показателей, которые покажут, сколько в среднем потребители готовы платить за услуги нашего сервиса доставки. Остановимся на следующих метриках:

1. ARPU (Average Revenue Per User) — средняя выручка на одного пользователя за определённый период.

2. ARPPU (Average Revenue Per Paying User) — средняя выручка на одного платящего пользователя за определённый период.

3. AOV (Average Order Value) — средний чек, или отношение выручки за определённый период к общему количеству заказов за это же время.

Задание:

Для каждого дня в таблицах orders и user_actions рассчитайте следующие показатели:

Выручку на пользователя (ARPU) за текущий день.
Выручку на платящего пользователя (ARPPU) за текущий день.
Выручку с заказа, или средний чек (AOV) за текущий день.
Колонки с показателями назовите соответственно arpu, arppu, aov. Колонку с датами назовите date. 

При расчёте всех показателей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты. 

Поля в результирующей таблице: date, arpu, arppu, aov

Пояснение: 

Будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

Платящими будем считать тех пользователей, которые в данный день оформили хотя бы один заказ, который в дальнейшем не был отменен.

При расчёте выручки помните, что не все заказы были оплачены — некоторые были отменены пользователями.

Не забывайте при делении заранее приводить значения к нужному типу данных.



```sql
SELECT date,
       arpu,
       arppu,
       aov
FROM   (SELECT date,
               round(revenue::decimal / oll_user_count, 2) as arpu,
               round(revenue::decimal / paying_user_count, 2) as arppu
        FROM   (SELECT date,
                       sum(price) as revenue
                FROM   (SELECT order_id,
                               creation_time::date as date,
                               unnest(product_ids) as product_id
                        FROM   orders
                        WHERE  order_id not in (SELECT order_id
                                                FROM   user_actions
                                                WHERE  action = 'cancel_order')) as t1
                    LEFT JOIN products using(product_id)
                GROUP BY date) as t11
            LEFT JOIN (SELECT count(distinct user_id) as oll_user_count,
                              count(distinct user_id) filter(WHERE order_id not in (SELECT order_id
                                                                             FROM   user_actions
                                                                             WHERE  action = 'cancel_order')) as paying_user_count, time::date as date
                       FROM   user_actions
                       GROUP BY date) as t2 using(date)) as ttt
    LEFT JOIN (SELECT round(avg(revenue), 2) as aov,
                      date
               FROM   (SELECT order_id,
                              date,
                              sum(price) as revenue
                       FROM   (SELECT order_id,
                                      creation_time::date as date,
                                      unnest(product_ids) as product_id
                               FROM   orders
                               WHERE  order_id not in (SELECT order_id
                                                       FROM   user_actions
                                                       WHERE  action = 'cancel_order')) as t1
                           LEFT JOIN products using(product_id)
                       GROUP BY order_id, date) as q1
               GROUP BY date) as q2 using(date)
ORDER BY date
```

# *Задача 3.

Дополним наш анализ ещё более интересными расчётами — вычислим все те же метрики, но для каждого дня будем учитывать накопленную выручку и все имеющиеся на текущий момент данные о числе пользователей и заказов. Таким образом, получим динамический ARPU, ARPPU и AOV и сможем проследить, как он менялся на протяжении времени с учётом поступающих нам данных.

Задание:

По таблицам orders и user_actions для каждого дня рассчитайте следующие показатели:

Накопленную выручку на пользователя (Running ARPU).
Накопленную выручку на платящего пользователя (Running ARPPU).
Накопленную выручку с заказа, или средний чек (Running AOV).
Колонки с показателями назовите соответственно running_arpu, running_arppu, running_aov. Колонку с датами назовите date. 

При расчёте всех показателей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты. 

Поля в результирующей таблице: date, running_arpu, running_arppu, running_aov

Пояснение: 

При расчёте числа пользователей и платящих пользователей на текущую дату учитывайте соответствующих пользователей за все предыдущие дни, включая текущий.

Платящими будем считать тех пользователей, которые на текущий день оформили хотя бы один заказ, который в дальнейшем не был отменен.

Будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

При расчёте выручки помните, что не все заказы были оплачены — некоторые были отменены пользователями.

Не забывайте при делении заранее приводить значения к нужному типу данных.

```sql
SELECT date,
       running_arpu,
       running_arppu,
       running_aov
FROM   (SELECT date,
               sum(revenue) OVER(ORDER BY date) as sum_sum_price,
               round(sum(revenue) OVER(ORDER BY date)::decimal / sum(oll_user_count) OVER(ORDER BY date),
                     2) as running_arpu,
               round(sum(revenue) OVER(ORDER BY date)::decimal / sum(paying_user_count) OVER(ORDER BY date),
                     2) as running_arppu
        FROM   (SELECT date,
                       sum(price) as revenue
                FROM   (SELECT order_id,
                               creation_time::date as date,
                               unnest(product_ids) as product_id
                        FROM   orders
                        WHERE  order_id not in (SELECT order_id
                                                FROM   user_actions
                                                WHERE  action = 'cancel_order')) as t1
                    LEFT JOIN products using(product_id)
                GROUP BY date) as t11
            LEFT JOIN ((SELECT count(distinct user_id) as oll_user_count,
                               time::date as date
                        FROM   (SELECT user_id,
                                       min(time) as time
                                FROM   user_actions
                                GROUP BY user_id) as r1
                        GROUP BY date) as t2
            LEFT JOIN (SELECT count(distinct user_id) as paying_user_count,
                              time::date as date
                       FROM   (SELECT user_id,
                                      min(time) as time
                               FROM   user_actions
                               WHERE  order_id not in (SELECT order_id
                                                       FROM   user_actions
                                                       WHERE  action = 'cancel_order')
                               GROUP BY user_id) as r1
                       GROUP BY date) as t5 using(date)) as o1 using(date)) as ttt
    LEFT JOIN (SELECT round(avg(running_aov), 2) as running_aov,
                      date
               FROM   (SELECT round(avg(revenue) OVER(ORDER BY date), 2) as running_aov,
                              date
                       FROM   (SELECT order_id,
                                      date,
                                      sum(price) as revenue
                               FROM   (SELECT order_id,
                                              creation_time::date as date,
                                              unnest(product_ids) as product_id
                                       FROM   orders
                                       WHERE  order_id not in (SELECT order_id
                                                               FROM   user_actions
                                                               WHERE  action = 'cancel_order')) as t1
                                   LEFT JOIN products using(product_id)
                               GROUP BY order_id, date) as q1
                       ORDER BY date) as q2
               GROUP BY date) as qqqqq using(date)
ORDER BY date
```

# Задача 4.

Давайте посчитаем те же показатели, но в другом разрезе — не просто по дням, а по дням недели.

Задание:

Для каждого дня недели в таблицах orders и user_actions рассчитайте следующие показатели:

Выручку на пользователя (ARPU).
Выручку на платящего пользователя (ARPPU).
Выручку на заказ (AOV).
При расчётах учитывайте данные только за период с 26 августа 2022 года по 8 сентября 2022 года включительно — так, чтобы в анализ попало одинаковое количество всех дней недели (ровно по два дня).

В результирующую таблицу включите как наименования дней недели (например, Monday), так и порядковый номер дня недели (от 1 до 7, где 1 — это Monday, 7 — это Sunday).

Колонки с показателями назовите соответственно arpu, arppu, aov. Колонку с наименованием дня недели назовите weekday, а колонку с порядковым номером дня недели weekday_number.

При расчёте всех показателей округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию порядкового номера дня недели.

Поля в результирующей таблице: 

weekday, weekday_number, arpu, arppu, aov

Пояснение: 

Будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

Платящими будем считать тех пользователей, которые в данный день оформили хотя бы один заказ, который в дальнейшем не был отменен.

При расчёте выручки помните, что не все заказы были оплачены — некоторые были отменены пользователями.

Не забывайте при делении заранее приводить значения к нужному типу данных.

В этой задаче порядковый номер дня недели необходим для того, чтобы дни недели были расположены на графике слева направо в правильном порядке — не по возрастанию наименования, а по возрастанию порядкового номера. Для получения корректной визуализации в настройках оси X необходимо отключить сортировку, установленную по умолчанию.

```sql
SELECT weekday,
       weekday_number,
       round(day_price::decimal / users_count, 2) as arpu,
       round(day_price::decimal / paying_orders_count, 2) as arppu,
       round(day_price::decimal / orders_count, 2) as aov
FROM   (SELECT sum(price) as day_price ,
               orders_count ,
               weekday
        FROM   ((SELECT to_char(creation_time::date, 'Day') as weekday,
                        count(order_id) OVER(PARTITION BY to_char(creation_time::date, 'Day')) as orders_count,
                        unnest(product_ids) as product_id
                 FROM   orders
                 WHERE  order_id not in (SELECT order_id
                                         FROM   user_actions
                                         WHERE  action = 'cancel_order')
                    and creation_time >= '2022-08-26'
                    and creation_time <= '2022-09-09') as t1
            LEFT JOIN products using(product_id)) as t11
        GROUP BY orders_count, weekday
    LEFT JOIN (SELECT count(distinct user_id) as users_count,
                      count(distinct user_id) filter(WHERE order_id not in (SELECT order_id
                                                                     FROM   user_actions
                                                                     WHERE  action = 'cancel_order')) as paying_orders_count, to_char(time, 'Day') as weekday, date_part('isodow', time) as weekday_number
               FROM   user_actions
               WHERE  time >= '2022-08-26'
                  and time <= '2022-09-09'
               GROUP BY weekday, weekday_number) as counts_users using(weekday)
ORDER BY weekday_number
```

# *Задача 5.

Немного усложним наш первоначальный запрос и отдельно посчитаем ежедневную выручку с заказов новых пользователей нашего сервиса. Посмотрим, какую долю она составляет в общей выручке с заказов всех пользователей — и новых, и старых.

Задание:

Для каждого дня в таблицах orders и user_actions рассчитайте следующие показатели:

Выручку, полученную в этот день.
Выручку с заказов новых пользователей, полученную в этот день.
Долю выручки с заказов новых пользователей в общей выручке, полученной за этот день.
Долю выручки с заказов остальных пользователей в общей выручке, полученной за этот день.
Колонки с показателями назовите соответственно revenue, new_users_revenue, new_users_revenue_share, old_users_revenue_share. Колонку с датами назовите date. 

Все показатели долей необходимо выразить в процентах. При их расчёте округляйте значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице:

date, revenue, new_users_revenue, new_users_revenue_share, old_users_revenue_share

Пояснение: 

Будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

Новыми будем считать тех пользователей, которые в данный день совершили своё первое действие в нашем сервисе.

При расчёте выручки помните, что не все заказы были оплачены — некоторые были отменены пользователями.

Не забывайте при делении заранее приводить значения к нужному типу данных.

```sql
SELECT date,
       revenue,
       new_users_revenue,
       round(new_users_revenue / revenue * 100, 2) as new_users_revenue_share,
       100 - round(new_users_revenue / revenue * 100, 2) as old_users_revenue_share
FROM   (SELECT creation_time::date as date,
               sum(price) as revenue
        FROM   (SELECT order_id,
                       creation_time,
                       unnest(product_ids) as product_id
                FROM   orders
                WHERE  order_id not in (SELECT order_id
                                        FROM   user_actions
                                        WHERE  action = 'cancel_order')) t3
            LEFT JOIN products using (product_id)
        GROUP BY date) t1
    LEFT JOIN (SELECT start_date as date,
                      sum(revenue) as new_users_revenue
               FROM   (SELECT t5.user_id,
                              t5.start_date,
                              coalesce(t6.revenue, 0) as revenue
                       FROM   (SELECT user_id,
                                      min(time::date) as start_date
                               FROM   user_actions
                               GROUP BY user_id) t5
                           LEFT JOIN (SELECT user_id,
                                             date,
                                             sum(order_price) as revenue
                                      FROM   (SELECT user_id,
                                                     time::date as date,
                                                     order_id
                                              FROM   user_actions
                                              WHERE  order_id not in (SELECT order_id
                                                                      FROM   user_actions
                                                                      WHERE  action = 'cancel_order')) t7
                                          LEFT JOIN (SELECT order_id,
                                                            sum(price) as order_price
                                                     FROM   (SELECT order_id,
                                                                    unnest(product_ids) as product_id
                                                             FROM   orders
                                                             WHERE  order_id not in (SELECT order_id
                                                                                     FROM   user_actions
                                                                                     WHERE  action = 'cancel_order')) t9
                                                         LEFT JOIN products using (product_id)
                                                     GROUP BY order_id) t8 using (order_id)
                                      GROUP BY user_id, date) t6
                               ON t5.user_id = t6.user_id and
                                  t5.start_date = t6.date) t4
               GROUP BY start_date) t2 using (date)
```

# Задача 6.

Также было бы интересно посмотреть, какие товары пользуются наибольшим спросом и приносят нам основной доход.

Задание:

Для каждого товара, представленного в таблице products, за весь период времени в таблице orders рассчитайте следующие показатели:

Суммарную выручку, полученную от продажи этого товара за весь период.
Долю выручки от продажи этого товара в общей выручке, полученной за весь период.
Колонки с показателями назовите соответственно revenue и share_in_revenue. Колонку с наименованиями товаров назовите product_name.

Долю выручки с каждого товара необходимо выразить в процентах. При её расчёте округляйте значения до двух знаков после запятой.

Товары, округлённая доля которых в выручке составляет менее 0.5%, объедините в общую группу с названием «ДРУГОЕ» (без кавычек), просуммировав округлённые доли этих товаров.

Результат должен быть отсортирован по убыванию выручки от продажи товара.

Поля в результирующей таблице: product_name, revenue, share_in_revenue

Пояснение: 

Будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

При расчёте выручки помните, что не все заказы были оплачены — некоторые были отменены пользователями.

Товары с небольшой долей в выручке необходимо объединить в одну группу, чтобы не выводить на графике абсолютно все товары из таблицы products.



```sql
SELECT product_name,
       sum(revenue) as revenue,
       sum(share_in_revenue) as share_in_revenue
FROM   (
SELECT case when round(100 * revenue::decimal / sum(revenue) OVER(),
                       2) < 0.5 then 'ДРУГОЕ'
            else name end as product_name ,
       revenue ,
       round(100 * revenue::decimal / sum(revenue) OVER(), 2) as share_in_revenue FROM(
SELECT name,
       sum(price) as revenue
FROM   ((
SELECT unnest(product_ids) as product_id
FROM   orders
WHERE  order_id not in (
SELECT order_id
FROM   user_actions
WHERE  action = 'cancel_order')) as t1
    LEFT JOIN (
SELECT name,
       price,
       product_id
FROM   products) as t2 using(product_id)) as t12
GROUP BY name) as t120
ORDER BY revenue desc) as t10000
GROUP BY product_name
ORDER BY revenue desc
```

# *Задача 7.
Теперь попробуем учесть в наших расчётах затраты с налогами и посчитаем валовую прибыль, то есть ту сумму, которую мы фактически получили в результате реализации товаров за рассматриваемый период.

Задание:

Для каждого дня в таблицах orders и courier_actions рассчитайте следующие показатели:

Выручку, полученную в этот день.
Затраты, образовавшиеся в этот день.
Сумму НДС с продажи товаров в этот день.
Валовую прибыль в этот день (выручка за вычетом затрат и НДС).
Суммарную выручку на текущий день.
Суммарные затраты на текущий день.
Суммарный НДС на текущий день.
Суммарную валовую прибыль на текущий день.
Долю валовой прибыли в выручке за этот день (долю п.4 в п.1).
Долю суммарной валовой прибыли в суммарной выручке на текущий день (долю п.8 в п.5).
Колонки с показателями назовите соответственно revenue, costs, tax, gross_profit, total_revenue, total_costs, total_tax, total_gross_profit, gross_profit_ratio, total_gross_profit_ratio

Колонку с датами назовите date.

Долю валовой прибыли в выручке необходимо выразить в процентах, округлив значения до двух знаков после запятой.

Результат должен быть отсортирован по возрастанию даты.

Поля в результирующей таблице: date, revenue, costs, tax, gross_profit, total_revenue, total_costs, total_tax, total_gross_profit, gross_profit_ratio,total_gross_profit_ratio

Чтобы посчитать затраты, в этой задаче введём дополнительные условия.

В упрощённом виде затраты нашего сервиса будем считать как сумму постоянных и переменных издержек. К постоянным издержкам отнесём аренду складских помещений, а к переменным — стоимость сборки и доставки заказа. Таким образом, переменные затраты будут напрямую зависеть от числа заказов.

Из данных, которые нам предоставил финансовый отдел, известно, что в августе 2022 года постоянные затраты составляли 120 000 рублей в день. Однако уже в сентябре нашему сервису потребовались дополнительные помещения, и поэтому постоянные затраты возросли до 150 000 рублей в день.

Также известно, что в августе 2022 года сборка одного заказа обходилась нам в 140 рублей, при этом курьерам мы платили по 150 рублей за один доставленный заказ и ещё 400 рублей ежедневно в качестве бонуса, если курьер доставлял не менее 5 заказов в день. В сентябре продакт-менеджерам удалось снизить затраты на сборку заказа до 115 рублей, но при этом пришлось повысить бонусную выплату за доставку 5 и более заказов до 500 рублей, чтобы обеспечить более конкурентоспособные условия труда. При этом в сентябре выплата курьерам за один доставленный заказ осталась неизменной.

Пояснение: 

При расчёте переменных затрат учитывайте следующие условия:

1. Затраты на сборку учитываются в том же дне, когда был оформлен заказ. Сборка отменённых заказов не производится.

2. Выплата курьерам за доставленный заказ начисляется сразу же после его доставки, поэтому если курьер доставит заказ на следующий день, то и выплата будет учтена в следующем дне.

3. Для получения бонусной выплаты курьерам необходимо доставить не менее 5 заказов в течение одного дня, поэтому если курьер примет 5 заказов в течение дня, но последний из них доставит после полуночи, бонусную выплату он не получит.

При расчёте НДС учитывайте, что для некоторых товаров налог составляет 10%, а не 20%. Список товаров со сниженным НДС:

'сахар', 'сухарики', 'сушки', 'семечки', 
'масло льняное', 'виноград', 'масло оливковое', 
'арбуз', 'батон', 'йогурт', 'сливки', 'гречка', 
'овсянка', 'макароны', 'баранина', 'апельсины', 
'бублики', 'хлеб', 'горох', 'сметана', 'рыба копченая', 
'мука', 'шпроты', 'сосиски', 'свинина', 'рис', 
'масло кунжутное', 'сгущенка', 'ананас', 'говядина', 
'соль', 'рыба вяленая', 'масло подсолнечное', 'яблоки', 
'груши', 'лепешка', 'молоко', 'курица', 'лаваш', 'вафли', 'мандарины'


Также при расчёте величины НДС по каждому товару округляйте значения до двух знаков после запятой.

При расчёте выручки по-прежнему будем считать, что оплата за заказ поступает сразу же после его оформления, т.е. случаи, когда заказ был оформлен в один день, а оплата получена на следующий, возникнуть не могут.

Также помните, что не все заказы были оплачены — некоторые были отменены пользователями.

```sql
with costs as (
  SELECT 
    date, 
    case when date_part ('month', date) = 8 
    and date_part ('year', date) = 2022 then 120000 + sum (order_assembly_costs) + sum (delivery_order_costs) + sum (delivery_orders_bonus) else 150000 + sum (order_assembly_costs) + sum (delivery_order_costs) + sum (delivery_orders_bonus) end as costs 
  FROM 
    (
      SELECT 
        date, 
        courier_id, 
        delivery_orders, 
        accepted_orders, 
        case when date_part ('month', date) = 8 
        and date_part ('year', date) = 2022 then accepted_orders * 140 else accepted_orders * 115 end as order_assembly_costs, 
        delivery_orders * 150 as delivery_order_costs, 
        case when date_part ('month', date) = 8 
        and date_part ('year', date) = 2022 
        and delivery_orders >= 5 then 400 when date_part ('month', date) = 9 
        and date_part ('year', date) = 2022 
        and delivery_orders >= 5 then 500 else 0 end as delivery_orders_bonus 
      FROM 
        (
          SELECT 
            date(time) as date, 
            courier_id, 
            count (order_id) filter (
              WHERE 
                action = 'deliver_order'
            ) as delivery_orders, 
            count (order_id) filter (
              WHERE 
                action = 'accept_order'
            ) as accepted_orders 
          FROM 
            courier_actions 
          WHERE 
            order_id not in (
              SELECT 
                order_id 
              FROM 
                user_actions 
              WHERE 
                action = 'cancel_order'
            ) 
          GROUP BY 
            date, 
            courier_id
        ) t
    ) t1 
  GROUP BY 
    date
), 
revenue as (
  SELECT 
    date, 
    sum (price) as revenue, 
    sum(product_tax) as tax 
  FROM 
    (
      SELECT 
        date, 
        order_id, 
        name, 
        price, 
        case when name in (
          'сахар', 'сухарики', 
          'сушки', 'семечки', 
          'масло льняное', 'виноград', 
          'масло оливковое', 
          'арбуз', 'батон', 'йогурт', 
          'сливки', 'гречка', 
          'овсянка', 'макароны', 
          'баранина', 'апельсины', 
          'бублики', 'хлеб', 'горох', 
          'сметана', 'рыба копченая', 
          'мука', 'шпроты', 'сосиски', 
          'свинина', 'рис', 'масло кунжутное', 
          'сгущенка', 'ананас', 
          'говядина', 'соль', 
          'рыба вяленая', 'масло подсолнечное', 
          'яблоки', 'груши', 'лепешка', 
          'молоко', 'курица', 
          'лаваш', 'вафли', 'мандарины'
        ) then round (price * 10 / 110, 2) else round (price * 20 / 120, 2) end as product_tax 
      FROM 
        (
          SELECT 
            date(creation_time) as date, 
            order_id, 
            unnest(product_ids) as product_id 
          FROM 
            orders 
          WHERE 
            order_id not in (
              SELECT 
                order_id 
              FROM 
                user_actions 
              WHERE 
                action = 'cancel_order'
            )
        ) t1 
        LEFT JOIN products using (product_id)
    ) t 
  GROUP BY 
    date
) 
SELECT 
  date, 
  revenue, 
  costs, 
  tax, 
  revenue - costs - tax as gross_profit, 
  sum (revenue) OVER (
    ORDER BY 
      date
  ) as total_revenue, 
  sum (costs) OVER (
    ORDER BY 
      date
  ) as total_costs, 
  sum (tax) OVER (
    ORDER BY 
      date
  ) as total_tax, 
  sum (revenue - costs - tax) OVER (
    ORDER BY 
      date
  ) as total_gross_profit, 
  round (
    100 *(revenue - costs - tax)/ revenue :: decimal, 
    2
  ) as gross_profit_ratio, 
  round (
    100 * sum (revenue - costs - tax) OVER (
      ORDER BY 
        date
    )/ sum (revenue) OVER (
      ORDER BY 
        date
    ):: decimal, 
    2
  ) as total_gross_profit_ratio 
FROM 
  revenue 
  join costs using (date) 
ORDER BY 
  date
```

# Маркетинговые метрики

# Задача 1.

Познакомимся с метриками, которые помогают оценить эффективность нашего маркетинга — то, насколько эффективно мы привлекаем пользователей в наше приложение. 

Сперва рассмотрим метрику CAC (Customer Acquisition Cost), которая отражает затраты на привлечение одного покупателя.

Задание:

На основе таблицы user_actions рассчитайте метрику CAC для двух рекламных кампаний.

Список id пользователей, пришедших после рекламной кампании № 1:

8631, 8632, 8638, 8643, 8657, 8673, 8706, 8707, 8715, 8723, 8732, 8739, 8741, 
8750, 8751, 8752, 8770, 8774, 8788, 8791, 8804, 8810, 8815, 8828, 8830, 8845, 
8853, 8859, 8867, 8869, 8876, 8879, 8883, 8896, 8909, 8911, 8933, 8940, 8972, 
8976, 8988, 8990, 9002, 9004, 9009, 9019, 9020, 9035, 9036, 9061, 9069, 9071, 
9075, 9081, 9085, 9089, 9108, 9113, 9144, 9145, 9146, 9162, 9165, 9167, 9175, 
9180, 9182, 9197, 9198, 9210, 9223, 9251, 9257, 9278, 9287, 9291, 9313, 9317, 
9321, 9334, 9351, 9391, 9398, 9414, 9420, 9422, 9431, 9450, 9451, 9454, 9472, 
9476, 9478, 9491, 9494, 9505, 9512, 9518, 9524, 9526, 9528, 9531, 9535, 9550, 
9559, 9561, 9562, 9599, 9603, 9605, 9611, 9612, 9615, 9625, 9633, 9652, 9654, 
9655, 9660, 9662, 9667, 9677, 9679, 9689, 9695, 9720, 9726, 9739, 9740, 9762, 
9778, 9786, 9794, 9804, 9810, 9813, 9818, 9828, 9831, 9836, 9838, 9845, 9871, 
9887, 9891, 9896, 9897, 9916, 9945, 9960, 9963, 9965, 9968, 9971, 9993, 9998, 
9999, 10001, 10013, 10016, 10023, 10030, 10051, 10057, 10064, 10082, 10103, 
10105, 10122, 10134, 10135


Список id пользователей, пришедших после рекламной кампании № 2:

8629, 8630, 8644, 8646, 8650, 8655, 8659, 8660, 8663, 8665, 8670, 8675, 8680, 8681, 
8682, 8683, 8694, 8697, 8700, 8704, 8712, 8713, 8719, 8729, 8733, 8742, 8748, 8754, 
8771, 8794, 8795, 8798, 8803, 8805, 8806, 8812, 8814, 8825, 8827, 8838, 8849, 8851, 
8854, 8855, 8870, 8878, 8882, 8886, 8890, 8893, 8900, 8902, 8913, 8916, 8923, 8929, 
8935, 8942, 8943, 8949, 8953, 8955, 8966, 8968, 8971, 8973, 8980, 8995, 8999, 9000, 
9007, 9013, 9041, 9042, 9047, 9064, 9068, 9077, 9082, 9083, 9095, 9103, 9109, 9117, 
9123, 9127, 9131, 9137, 9140, 9149, 9161, 9179, 9181, 9183, 9185, 9190, 9196, 9203, 
9207, 9226, 9227, 9229, 9230, 9231, 9250, 9255, 9259, 9267, 9273, 9281, 9282, 9289, 
9292, 9303, 9310, 9312, 9315, 9327, 9333, 9335, 9337, 9343, 9356, 9368, 9370, 9383, 
9392, 9404, 9410, 9421, 9428, 9432, 9437, 9468, 9479, 9483, 9485, 9492, 9495, 9497, 
9498, 9500, 9510, 9527, 9529, 9530, 9538, 9539, 9545, 9557, 9558, 9560, 9564, 9567, 
9570, 9591, 9596, 9598, 9616, 9631, 9634, 9635, 9636, 9658, 9666, 9672, 9684, 9692, 
9700, 9704, 9706, 9711, 9719, 9727, 9735, 9741, 9744, 9749, 9752, 9753, 9755, 9757, 
9764, 9783, 9784, 9788, 9790, 9808, 9820, 9839, 9841, 9843, 9853, 9855, 9859, 9863, 
9877, 9879, 9880, 9882, 9883, 9885, 9901, 9904, 9908, 9910, 9912, 9920, 9929, 9930, 
9935, 9939, 9958, 9959, 9961, 9983, 10027, 10033, 10038, 10045, 10047, 10048, 10058, 
10059, 10067, 10069, 10073, 10075, 10078, 10079, 10081, 10092, 10106, 10110, 10113, 10131


Колонку с наименованиями кампаний назовите ads_campaign, а колонку со значением метрики — cac.

Наименования кампаний выведите в следующем виде:

Кампания № 1
Кампания № 2


Полученные значения метрики округлите до двух знаков после запятой.

Результат должен быть отсортирован по убыванию значения метрики.

Поля в результирующей таблице: ads_campaign, cac

```sql
with september_new_users as (SELECT user_id,
                                    order_id,
                                    case when user_id in (8631, 8632, 8638, 8643, 8657, 8673, 8706, 8707, 8715, 8723, 8732,
                                                          8739, 8741, 8750, 8751, 8752, 8770, 8774, 8788, 8791,
                                                          8804, 8810, 8815, 8828, 8830, 8845, 8853, 8859, 8867,
                                                          8869, 8876, 8879, 8883, 8896, 8909, 8911, 8933, 8940,
                                                          8972, 8976, 8988, 8990, 9002, 9004, 9009, 9019, 9020,
                                                          9035, 9036, 9061, 9069, 9071, 9075, 9081, 9085, 9089,
                                                          9108, 9113, 9144, 9145, 9146, 9162, 9165, 9167, 9175,
                                                          9180, 9182, 9197, 9198, 9210, 9223, 9251, 9257, 9278,
                                                          9287, 9291, 9313, 9317, 9321, 9334, 9351, 9391, 9398,
                                                          9414, 9420, 9422, 9431, 9450, 9451, 9454, 9472, 9476,
                                                          9478, 9491, 9494, 9505, 9512, 9518, 9524, 9526, 9528,
                                                          9531, 9535, 9550, 9559, 9561, 9562, 9599, 9603, 9605,
                                                          9611, 9612, 9615, 9625, 9633, 9652, 9654, 9655, 9660,
                                                          9662, 9667, 9677, 9679, 9689, 9695, 9720, 9726, 9739,
                                                          9740, 9762, 9778, 9786, 9794, 9804, 9810, 9813, 9818,
                                                          9828, 9831, 9836, 9838, 9845, 9871, 9887, 9891, 9896,
                                                          9897, 9916, 9945, 9960, 9963, 9965, 9968, 9971, 9993,
                                                          9998, 9999, 10001, 10013, 10016, 10023, 10030, 10051,
                                                          10057, 10064, 10082, 10103, 10105, 10122, 10134, 10135) then 'Кампания № 1'
                                         when user_id in (8629, 8630, 8644, 8646, 8650, 8655, 8659, 8660, 8663, 8665, 8670,
                                                          8675, 8680, 8681, 8682, 8683, 8694, 8697, 8700, 8704,
                                                          8712, 8713, 8719, 8729, 8733, 8742, 8748, 8754, 8771,
                                                          8794, 8795, 8798, 8803, 8805, 8806, 8812, 8814, 8825,
                                                          8827, 8838, 8849, 8851, 8854, 8855, 8870, 8878, 8882,
                                                          8886, 8890, 8893, 8900, 8902, 8913, 8916, 8923, 8929,
                                                          8935, 8942, 8943, 8949, 8953, 8955, 8966, 8968, 8971,
                                                          8973, 8980, 8995, 8999, 9000, 9007, 9013, 9041, 9042,
                                                          9047, 9064, 9068, 9077, 9082, 9083, 9095, 9103, 9109,
                                                          9117, 9123, 9127, 9131, 9137, 9140, 9149, 9161, 9179,
                                                          9181, 9183, 9185, 9190, 9196, 9203, 9207, 9226, 9227,
                                                          9229, 9230, 9231, 9250, 9255, 9259, 9267, 9273, 9281,
                                                          9282, 9289, 9292, 9303, 9310, 9312, 9315, 9327, 9333,
                                                          9335, 9337, 9343, 9356, 9368, 9370, 9383, 9392, 9404,
                                                          9410, 9421, 9428, 9432, 9437, 9468, 9479, 9483, 9485,
                                                          9492, 9495, 9497, 9498, 9500, 9510, 9527, 9529, 9530,
                                                          9538, 9539, 9545, 9557, 9558, 9560, 9564, 9567, 9570,
                                                          9591, 9596, 9598, 9616, 9631, 9634, 9635, 9636, 9658,
                                                          9666, 9672, 9684, 9692, 9700, 9704, 9706, 9711, 9719,
                                                          9727, 9735, 9741, 9744, 9749, 9752, 9753, 9755, 9757,
                                                          9764, 9783, 9784, 9788, 9790, 9808, 9820, 9839, 9841,
                                                          9843, 9853, 9855, 9859, 9863, 9877, 9879, 9880, 9882,
                                                          9883, 9885, 9901, 9904, 9908, 9910, 9912, 9920, 9929,
                                                          9930, 9935, 9939, 9958, 9959, 9961, 9983, 10027, 10033,
                                                          10038, 10045, 10047, 10048, 10058, 10059, 10067, 10069,
                                                          10073, 10075, 10078, 10079, 10081, 10092, 10106, 10110,
                                                          10113, 10131) then 'Кампания № 2'
                                         else null end as ads_campaign
                             FROM   user_actions
                             WHERE  order_id not in (SELECT order_id
                                                     FROM   user_actions
                                                     WHERE  action = 'cancel_order'))
SELECT ads_campaign,
       round (250000/count (distinct user_id)::decimal, 2) as cac
FROM   (SELECT user_id,
               ads_campaign
        FROM   september_new_users
        WHERE  ads_campaign is not null
        GROUP BY user_id, ads_campaign) t
GROUP BY ads_campaign
ORDER BY cac desc
```

# Задача 2.

Задание:

Рассчитайте ROI для каждого рекламного канала.

Колонку с наименованиями кампаний назовите ads_campaign, а колонку со значением метрики — roi.

Наименования кампаний выведите в следующем виде:

Кампания № 1

Кампания № 2


Полученные значения метрики необходимо выразить в процентах и округлить до двух знаков после запятой.

Результат должен быть отсортирован по убыванию значения метрики.

Поля в результирующей таблице: ads_campaign, roi

```sql
with main_t as(SELECT user_id,
                      order_id,
                      action,
                      case when user_id in (8631, 8632, 8638, 8643, 8657, 8673, 8706, 8707, 8715, 8723, 8732,
                                            8739, 8741, 8750, 8751, 8752, 8770, 8774, 8788, 8791,
                                            8804, 8810, 8815, 8828, 8830, 8845, 8853, 8859, 8867,
                                            8869, 8876, 8879, 8883, 8896, 8909, 8911, 8933, 8940,
                                            8972, 8976, 8988, 8990, 9002, 9004, 9009, 9019, 9020,
                                            9035, 9036, 9061, 9069, 9071, 9075, 9081, 9085, 9089,
                                            9108, 9113, 9144, 9145, 9146, 9162, 9165, 9167, 9175,
                                            9180, 9182, 9197, 9198, 9210, 9223, 9251, 9257, 9278,
                                            9287, 9291, 9313, 9317, 9321, 9334, 9351, 9391, 9398,
                                            9414, 9420, 9422, 9431, 9450, 9451, 9454, 9472, 9476,
                                            9478, 9491, 9494, 9505, 9512, 9518, 9524, 9526, 9528,
                                            9531, 9535, 9550, 9559, 9561, 9562, 9599, 9603, 9605,
                                            9611, 9612, 9615, 9625, 9633, 9652, 9654, 9655, 9660,
                                            9662, 9667, 9677, 9679, 9689, 9695, 9720, 9726, 9739,
                                            9740, 9762, 9778, 9786, 9794, 9804, 9810, 9813, 9818,
                                            9828, 9831, 9836, 9838, 9845, 9871, 9887, 9891, 9896,
                                            9897, 9916, 9945, 9960, 9963, 9965, 9968, 9971, 9993,
                                            9998, 9999, 10001, 10013, 10016, 10023, 10030, 10051,
                                            10057, 10064, 10082, 10103, 10105, 10122, 10134, 10135) then 1
                           when user_id in (8629, 8630, 8644, 8646, 8650, 8655, 8659, 8660, 8663, 8665, 8670,
                                            8675, 8680, 8681, 8682, 8683, 8694, 8697, 8700, 8704,
                                            8712, 8713, 8719, 8729, 8733, 8742, 8748, 8754, 8771,
                                            8794, 8795, 8798, 8803, 8805, 8806, 8812, 8814, 8825,
                                            8827, 8838, 8849, 8851, 8854, 8855, 8870, 8878, 8882,
                                            8886, 8890, 8893, 8900, 8902, 8913, 8916, 8923, 8929,
                                            8935, 8942, 8943, 8949, 8953, 8955, 8966, 8968, 8971,
                                            8973, 8980, 8995, 8999, 9000, 9007, 9013, 9041, 9042,
                                            9047, 9064, 9068, 9077, 9082, 9083, 9095, 9103, 9109,
                                            9117, 9123, 9127, 9131, 9137, 9140, 9149, 9161, 9179,
                                            9181, 9183, 9185, 9190, 9196, 9203, 9207, 9226, 9227,
                                            9229, 9230, 9231, 9250, 9255, 9259, 9267, 9273, 9281,
                                            9282, 9289, 9292, 9303, 9310, 9312, 9315, 9327, 9333,
                                            9335, 9337, 9343, 9356, 9368, 9370, 9383, 9392, 9404,
                                            9410, 9421, 9428, 9432, 9437, 9468, 9479, 9483, 9485,
                                            9492, 9495, 9497, 9498, 9500, 9510, 9527, 9529, 9530,
                                            9538, 9539, 9545, 9557, 9558, 9560, 9564, 9567, 9570,
                                            9591, 9596, 9598, 9616, 9631, 9634, 9635, 9636, 9658,
                                            9666, 9672, 9684, 9692, 9700, 9704, 9706, 9711, 9719,
                                            9727, 9735, 9741, 9744, 9749, 9752, 9753, 9755, 9757,
                                            9764, 9783, 9784, 9788, 9790, 9808, 9820, 9839, 9841,
                                            9843, 9853, 9855, 9859, 9863, 9877, 9879, 9880, 9882,
                                            9883, 9885, 9901, 9904, 9908, 9910, 9912, 9920, 9929,
                                            9930, 9935, 9939, 9958, 9959, 9961, 9983, 10027, 10033,
                                            10038, 10045, 10047, 10048, 10058, 10059, 10067, 10069,
                                            10073, 10075, 10078, 10079, 10081, 10092, 10106, 10110,
                                            10113, 10131) then 2
                           else 0 end as ads_campaign,
                      count(action) filter (WHERE action = 'cancel_order') OVER (PARTITION BY order_id) as is_canceled
               FROM   user_actions), order_price as (SELECT order_id,
                                             sum(price) as order_price
                                      FROM   (SELECT order_id,
                                                     unnest(product_ids) as product_id
                                              FROM   orders) t1
                                          LEFT JOIN products using (product_id)
                                      GROUP BY order_id)
SELECT concat('Кампания № ', ads_campaign) as ads_campaign,
       round((sum(order_price)-250000)/250000*100, 2) as roi
FROM   main_t
    LEFT JOIN order_price using (order_id)
WHERE  ads_campaign in (1, 2)
   and is_canceled = 0
GROUP BY ads_campaign
ORDER BY roi desc
```

# Задача 3.

Итак, мы выяснили, что первая рекламная кампания позволяет привлекать более качественных лидов. Но причина этих различий пока не совсем ясна. Почему пользователи из первого рекламного канала приносят нам больше денег? Может быть, у них выше средний чек? 

Давайте проведём более детальный анализ, чтобы выяснить, чем отличаются два рекламных канала с точки зрения пользовательских метрик.

Задание:

Для каждой рекламной кампании посчитайте среднюю стоимость заказа привлечённых пользователей за первую неделю использования приложения с 1 по 7 сентября 2022 года.

Колонку с наименованиями кампаний назовите ads_campaign, а колонку со значением метрики — avg_check.

Наименования кампаний выведите в следующем виде:

Кампания № 1
Кампания № 2


Полученные значения метрики необходимо округлить до двух знаков после запятой.

Результат должен быть отсортирован по убыванию значения метрики.

Поля в результирующей таблице: ads_campaign, avg_check

Пояснение: 

Покупателями будем считать тех пользователей, которые сделали хотя бы один заказ, который в дальнейшем не был отменён. Например, если человек сделал только один заказ, а потом отменил его, то покупателем мы его не считаем.

Не забывайте при делении заранее приводить значения к нужному типу данных.

Списки зарегистрировавшихся пользователей те же, что и на предыдущих шагах.

```sql
with main_t as(SELECT user_id,
                      order_id,
                      action,
                      order_price,
                      ads_campaign,
                      is_canceled
               FROM   (SELECT date(time) as date,
                              user_id,
                              order_id,
                              action,
                              case when user_id in (8631, 8632, 8638, 8643, 8657, 8673, 8706, 8707, 8715, 8723, 8732,
                                                    8739, 8741, 8750, 8751, 8752, 8770, 8774, 8788, 8791,
                                                    8804, 8810, 8815, 8828, 8830, 8845, 8853, 8859, 8867,
                                                    8869, 8876, 8879, 8883, 8896, 8909, 8911, 8933, 8940,
                                                    8972, 8976, 8988, 8990, 9002, 9004, 9009, 9019, 9020,
                                                    9035, 9036, 9061, 9069, 9071, 9075, 9081, 9085, 9089,
                                                    9108, 9113, 9144, 9145, 9146, 9162, 9165, 9167, 9175,
                                                    9180, 9182, 9197, 9198, 9210, 9223, 9251, 9257, 9278,
                                                    9287, 9291, 9313, 9317, 9321, 9334, 9351, 9391, 9398,
                                                    9414, 9420, 9422, 9431, 9450, 9451, 9454, 9472, 9476,
                                                    9478, 9491, 9494, 9505, 9512, 9518, 9524, 9526, 9528,
                                                    9531, 9535, 9550, 9559, 9561, 9562, 9599, 9603, 9605,
                                                    9611, 9612, 9615, 9625, 9633, 9652, 9654, 9655, 9660,
                                                    9662, 9667, 9677, 9679, 9689, 9695, 9720, 9726, 9739,
                                                    9740, 9762, 9778, 9786, 9794, 9804, 9810, 9813, 9818,
                                                    9828, 9831, 9836, 9838, 9845, 9871, 9887, 9891, 9896,
                                                    9897, 9916, 9945, 9960, 9963, 9965, 9968, 9971, 9993,
                                                    9998, 9999, 10001, 10013, 10016, 10023, 10030, 10051,
                                                    10057, 10064, 10082, 10103, 10105, 10122, 10134, 10135) then 1
                                   when user_id in (8629, 8630, 8644, 8646, 8650, 8655, 8659, 8660, 8663, 8665, 8670,
                                                    8675, 8680, 8681, 8682, 8683, 8694, 8697, 8700, 8704,
                                                    8712, 8713, 8719, 8729, 8733, 8742, 8748, 8754, 8771,
                                                    8794, 8795, 8798, 8803, 8805, 8806, 8812, 8814, 8825,
                                                    8827, 8838, 8849, 8851, 8854, 8855, 8870, 8878, 8882,
                                                    8886, 8890, 8893, 8900, 8902, 8913, 8916, 8923, 8929,
                                                    8935, 8942, 8943, 8949, 8953, 8955, 8966, 8968, 8971,
                                                    8973, 8980, 8995, 8999, 9000, 9007, 9013, 9041, 9042,
                                                    9047, 9064, 9068, 9077, 9082, 9083, 9095, 9103, 9109,
                                                    9117, 9123, 9127, 9131, 9137, 9140, 9149, 9161, 9179,
                                                    9181, 9183, 9185, 9190, 9196, 9203, 9207, 9226, 9227,
                                                    9229, 9230, 9231, 9250, 9255, 9259, 9267, 9273, 9281,
                                                    9282, 9289, 9292, 9303, 9310, 9312, 9315, 9327, 9333,
                                                    9335, 9337, 9343, 9356, 9368, 9370, 9383, 9392, 9404,
                                                    9410, 9421, 9428, 9432, 9437, 9468, 9479, 9483, 9485,
                                                    9492, 9495, 9497, 9498, 9500, 9510, 9527, 9529, 9530,
                                                    9538, 9539, 9545, 9557, 9558, 9560, 9564, 9567, 9570,
                                                    9591, 9596, 9598, 9616, 9631, 9634, 9635, 9636, 9658,
                                                    9666, 9672, 9684, 9692, 9700, 9704, 9706, 9711, 9719,
                                                    9727, 9735, 9741, 9744, 9749, 9752, 9753, 9755, 9757,
                                                    9764, 9783, 9784, 9788, 9790, 9808, 9820, 9839, 9841,
                                                    9843, 9853, 9855, 9859, 9863, 9877, 9879, 9880, 9882,
                                                    9883, 9885, 9901, 9904, 9908, 9910, 9912, 9920, 9929,
                                                    9930, 9935, 9939, 9958, 9959, 9961, 9983, 10027, 10033,
                                                    10038, 10045, 10047, 10048, 10058, 10059, 10067, 10069,
                                                    10073, 10075, 10078, 10079, 10081, 10092, 10106, 10110,
                                                    10113, 10131) then 2
                                   else 0 end as ads_campaign,
                              count(action) filter (WHERE action = 'cancel_order') OVER (PARTITION BY order_id) as is_canceled
                       FROM   user_actions) as t1
                   LEFT JOIN (SELECT order_id,
                                     sum(price) as order_price
                              FROM   (SELECT order_id,
                                             unnest(product_ids) as product_id
                                      FROM   orders) t
                                  LEFT JOIN products using (product_id)
                              GROUP BY order_id) as t2 using (order_id)
               WHERE  date between '2022-09-01'
                  and '2022-09-07 23:59:59')
SELECT concat('Кампания № ', ads_campaign) as ads_campaign,
       round (avg (avg_order_price), 2) as avg_check
FROM   (SELECT user_id,
               ads_campaign,
               is_canceled,
               avg (order_price) as avg_order_price
        FROM   main_t
        GROUP BY user_id, ads_campaign, is_canceled) t
WHERE  ads_campaign in (1, 2)
   and is_canceled = 0
GROUP BY ads_campaign
ORDER BY avg_check desc
```

# Задача 4.

Давайте сразу же закрепим на практике методику расчёта Retention и попробуем самостоятельно написать похожий запрос, руководствуясь материалами лекции.

Задание:

На основе данных в таблице user_actions рассчитайте показатель дневного Retention для всех пользователей, разбив их на когорты по дате первого взаимодействия с нашим приложением.

В результат включите четыре колонки: месяц первого взаимодействия, дату первого взаимодействия, количество дней, прошедших с даты первого взаимодействия (порядковый номер дня начиная с 0), и само значение Retention.

Колонки со значениями назовите соответственно start_month, start_date, day_number, retention.

Метрику необходимо выразить в виде доли, округлив полученные значения до двух знаков после запятой.

Месяц первого взаимодействия укажите в виде даты, округлённой до первого числа месяца.

Результат должен быть отсортирован сначала по возрастанию даты первого взаимодействия, затем по возрастанию порядкового номера дня.

Поля в результирующей таблице: start_month, start_date, day_number, retention

Пояснение: 

В этой задаче учитывайте всех уникальных пользователей в таблице user_actions.

Не забывайте при делении заранее приводить значения к нужному типу данных.

```sql
SELECT date_trunc('month', start_date)::date as start_month,
       start_date,
       date - start_date as day_number,
       round (count(distinct user_id)::decimal/ max(count(distinct user_id)) OVER (PARTITION BY start_date), 2) as retention
FROM   (SELECT user_id,
               min(time::date) OVER (PARTITION BY user_id) as start_date,
               time::date as date
        FROM   user_actions) t1
GROUP BY date, start_date
ORDER BY start_date, day_number
```

# Задача 5.

А теперь вернёмся к анализу рекламных кампаний и посчитаем Retention для двух групп пользователей. Возможно, дело именно в том, что один из каналов привёл более активных и заинтересованных пользователей. Давайте это выясним!

Задание:

Для каждой рекламной кампании посчитайте Retention 1-го и 7-го дня у привлечённых пользователей. 

В результат включите четыре колонки: колонку с наименованиями кампаний, дату первого взаимодействия с приложением, количество дней, прошедших с даты первого взаимодействия (порядковый номер), и само значение Retention.

Колонки со значениями назовите соответственно ads_campaign, start_date, day_number, retention.

Наименования кампаний выведите в следующем виде:

Кампания № 1

Кампания № 2


Метрику необходимо выразить в виде доли, округлив полученные значения до двух знаков после запятой.

Результат должен быть отсортирован сначала по наименованию кампании (по возрастанию), затем по возрастанию порядкового номера дня.

В результат должны попасть следующие дни: нулевой, первый и седьмой.

Поля в результирующей таблице: ads_campaign, start_date, day_number, retention

Пояснение: 

Не забывайте при делении заранее приводить значения к нужному типу данных.

Списки зарегистрировавшихся пользователей те же, что и на предыдущих шагах.

```sql
SELECT ads_campaign,
       start_date,
       day_number,
       round(count (distinct user_id)::decimal/ max(count (distinct user_id)) OVER (PARTITION BY ads_campaign),
             2) as retention
FROM   (SELECT user_id,
               ads_campaign,
               min(date) OVER (PARTITION BY user_id) as start_date,
               date,
               date - min(date) OVER (PARTITION BY user_id) as day_number
        FROM   (SELECT date(time) as date,
                       user_id,
                       case when user_id in (8631, 8632, 8638, 8643, 8657, 8673, 8706, 8707, 8715, 8723, 8732,
                                             8739, 8741, 8750, 8751, 8752, 8770, 8774, 8788, 8791,
                                             8804, 8810, 8815, 8828, 8830, 8845, 8853, 8859, 8867,
                                             8869, 8876, 8879, 8883, 8896, 8909, 8911, 8933, 8940,
                                             8972, 8976, 8988, 8990, 9002, 9004, 9009, 9019, 9020,
                                             9035, 9036, 9061, 9069, 9071, 9075, 9081, 9085, 9089,
                                             9108, 9113, 9144, 9145, 9146, 9162, 9165, 9167, 9175,
                                             9180, 9182, 9197, 9198, 9210, 9223, 9251, 9257, 9278,
                                             9287, 9291, 9313, 9317, 9321, 9334, 9351, 9391, 9398,
                                             9414, 9420, 9422, 9431, 9450, 9451, 9454, 9472, 9476,
                                             9478, 9491, 9494, 9505, 9512, 9518, 9524, 9526, 9528,
                                             9531, 9535, 9550, 9559, 9561, 9562, 9599, 9603, 9605,
                                             9611, 9612, 9615, 9625, 9633, 9652, 9654, 9655, 9660,
                                             9662, 9667, 9677, 9679, 9689, 9695, 9720, 9726, 9739,
                                             9740, 9762, 9778, 9786, 9794, 9804, 9810, 9813, 9818,
                                             9828, 9831, 9836, 9838, 9845, 9871, 9887, 9891, 9896,
                                             9897, 9916, 9945, 9960, 9963, 9965, 9968, 9971, 9993,
                                             9998, 9999, 10001, 10013, 10016, 10023, 10030, 10051,
                                             10057, 10064, 10082, 10103, 10105, 10122, 10134, 10135) then 'Кампания № 1'
                            when user_id in (8629, 8630, 8644, 8646, 8650, 8655, 8659, 8660, 8663, 8665, 8670,
                                             8675, 8680, 8681, 8682, 8683, 8694, 8697, 8700, 8704,
                                             8712, 8713, 8719, 8729, 8733, 8742, 8748, 8754, 8771,
                                             8794, 8795, 8798, 8803, 8805, 8806, 8812, 8814, 8825,
                                             8827, 8838, 8849, 8851, 8854, 8855, 8870, 8878, 8882,
                                             8886, 8890, 8893, 8900, 8902, 8913, 8916, 8923, 8929,
                                             8935, 8942, 8943, 8949, 8953, 8955, 8966, 8968, 8971,
                                             8973, 8980, 8995, 8999, 9000, 9007, 9013, 9041, 9042,
                                             9047, 9064, 9068, 9077, 9082, 9083, 9095, 9103, 9109,
                                             9117, 9123, 9127, 9131, 9137, 9140, 9149, 9161, 9179,
                                             9181, 9183, 9185, 9190, 9196, 9203, 9207, 9226, 9227,
                                             9229, 9230, 9231, 9250, 9255, 9259, 9267, 9273, 9281,
                                             9282, 9289, 9292, 9303, 9310, 9312, 9315, 9327, 9333,
                                             9335, 9337, 9343, 9356, 9368, 9370, 9383, 9392, 9404,
                                             9410, 9421, 9428, 9432, 9437, 9468, 9479, 9483, 9485,
                                             9492, 9495, 9497, 9498, 9500, 9510, 9527, 9529, 9530,
                                             9538, 9539, 9545, 9557, 9558, 9560, 9564, 9567, 9570,
                                             9591, 9596, 9598, 9616, 9631, 9634, 9635, 9636, 9658,
                                             9666, 9672, 9684, 9692, 9700, 9704, 9706, 9711, 9719,
                                             9727, 9735, 9741, 9744, 9749, 9752, 9753, 9755, 9757,
                                             9764, 9783, 9784, 9788, 9790, 9808, 9820, 9839, 9841,
                                             9843, 9853, 9855, 9859, 9863, 9877, 9879, 9880, 9882,
                                             9883, 9885, 9901, 9904, 9908, 9910, 9912, 9920, 9929,
                                             9930, 9935, 9939, 9958, 9959, 9961, 9983, 10027, 10033,
                                             10038, 10045, 10047, 10048, 10058, 10059, 10067, 10069,
                                             10073, 10075, 10078, 10079, 10081, 10092, 10106, 10110,
                                             10113, 10131) then 'Кампания № 2'
                            else null end as ads_campaign
                FROM   user_actions) t1
        WHERE  ads_campaign is not null) t2
WHERE  day_number in (0, 1, 7)
GROUP BY ads_campaign, start_date, day_number
```